# Writer Tools


This notebook provides a quick overview for getting started with Writer [tools](/docs//docs/concepts/tools/). For detailed documentation of all Writer features and configurations head to the [Writer docs](https://dev.writer.com/home).

## Overview

### Integration details

| Class                                                                                                                                       | Package          | Local | Serializable | JS support |                                        Package downloads                                         |                                        Package latest                                         |
|:--------------------------------------------------------------------------------------------------------------------------------------------|:-----------------| :---: | :---: |:----------:|:------------------------------------------------------------------------------------------------:|:---------------------------------------------------------------------------------------------:|
| [GraphTool](https://python.langchain.com/v0.2/api_reference/community/tools/langchain_community.tools.langchain_writer.tool.GraphTool.html) | [langchain-writer](https://api.python.langchain.com/en/latest/community_api_reference.html) |      ❌       |                                       ❌                                       | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-writer?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-writer?style=flat-square&label=%20) |

### Features

We provide usage of Writer [Knowledge Graph](https://dev.writer.com/api-guides/kg-chat) as a Lang Chain tool


## Setup

The integration lives in the `langchain-writer` package:

In [ ]:
%pip install --quiet -U langchain-writer

### Credentials

Head to [Writer AI Studio](https://app.writer.com/aistudio/signup?utm_campaign=devrel) to sign up and generate an API key. Once you've done this set the WRITER_API_KEY environment variable. It's necessary to create ChatWriter instance:

In [ ]:
import getpass
import os

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

In [ ]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Instantiation

Here we show how to instantiate an instance of the Writer Graph Tool, with

In [ ]:
from langchain_writer.tools import GraphTool

graph_id = getpass.getpass("Enter Writer Knowledge Graph ID: ")

tool = GraphTool(
    graph_ids = [graph_id]
)

## Invocation

### Invoke directly with args


In [ ]:
response = tool.invoke(
    {"question": "How to improve my physical conditions?"}
)
response

In [ ]:
print(response["answer"])

### Invoke with ToolCall

We can also invoke the tool with a model-generated ToolCall, in which case a ToolMessage will be returned:


In [ ]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {
            "question": "How to improve my physical conditions?",
        },
    "id": "1",
    "name": tool.name,
    "type": "tool_call",
}
response = tool.invoke(model_generated_tool_call)
response

## Async

All features above (direct invocation, invocation with tool call) also support asynchronous usage!

## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />


In [ ]:
from langchain_writer.chat_models import ChatWriter

llm = ChatWriter()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# specifying tool_choice will force the model to call this tool.
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)

llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


response = tool_chain.invoke("...")
response

## API reference
For detailed documentation of all GraphTool features and configurations head to the [API reference](https://python.langchain.com/api_reference/writer/tools/langchain_writer.tools.GraphTool.html#langchain_writer.tools.GraphTool)